<a href="https://colab.research.google.com/github/helgadenes/EPIC_4/blob/main/Day_1/Tutorial2/Tutorial2_project_star_formation_in_galaxies.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Analysing star formation properties of galaxies

### Project activities related to the 2nd tutorial in EPIC 4

This notebook aims to help you explore star formation in galaxies using data from the SDSS survey. The notebook provides instructions and some code to help you. The instructions ask you to perform some simple calculations and to create histograms and scatter plots.


First dwonload the relevant data from the Sloan DIgital Sky Survey (SDSS) data base. The code for this is provided. In the cell below.  

- The SpecObj table contains basic information about the observation and the object, such as the redshift.
- The galaxy table provides useful photometric measurements for colors and images.
- The three GalSpec table each give products derived from spectroscopy.

We are going to use the intensity measurments of hydrogen lines H$\alpha$ and H$\beta$ for the data analysis.

The H$\alpha$ emission line is a convienient indicator of the star formation rate.

We want galaxies with quality measurements of this line strength, so a S/N restriction is specified. We also want the H$\beta$ emission line, because as we will see below, this will assist us in accounting for dust absorption along our line of sight.

In [1]:
!pip install astroquery

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 5.3/5.3 MB 9.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 910.8/910.8 kB 18.4 MB/s eta 0:00:00


In [2]:
# Import relevant libraries

import pandas as pd # data analysis
import numpy as np # more data analysis

from matplotlib import pyplot as plt # another plotting library
from matplotlib.image import imread

from astropy.constants import c # useful constants
from astropy import units as u # useful for unit conversions
from astropy import coordinates as coords
from astropy.coordinates import SkyCoord
from astropy.io import fits
from astropy.io import ascii

from astroquery.sdss import SDSS
from astroquery.simbad import Simbad

# Python standard-libraries to download data from the web
from urllib.parse import urlencode
from urllib.request import urlretrieve

#only here to display images
from IPython.display import Image

In [3]:
# Create a query to fetch star forming galaxies

query = 'SELECT s.plate, s.fiberid, s.mjd, s.ra, s.dec, s.z, s.zwarning, '   # selecting the measured quantities that we want
query += 'g.h_beta_flux, g.h_alpha_flux, '
query += 'g.h_beta_flux_err, g.h_alpha_flux_err, '
query += 'e.bptclass, e.lgm_tot_p50, '
query += 'i.lick_hd_a_sub, i.d4000_n_sub, '
query += 'i.lick_hd_a_sub_err, i.d4000_n_sub_err, '
query += 'p.petror90_r, p.dered_u as u, p.dered_g as g '
query += 'FROM GalSpecLine AS g, SpecObj AS s, GalSpecExtra AS e, GalSpecIndx AS i, galaxy AS p '  # selecting the data tables
query += 'WHERE '                                  # matching tables and applying some selection criteria
query += 's.specobjid = g.specobjid '
query += 'AND s.specobjid = e.specobjid '
query += 'AND s.specobjid = i.specobjid '
query += 'AND s.bestobjid = p.objid '
query += 'AND bptclass = 1 '
query += 'AND p.petror90_r > 5 '
query += 'AND s.z BETWEEN 0.04 and 0.1 ' # Lower limit needed for global parameters w/ small aperture
query += 'AND h_alpha_flux > h_alpha_flux_err*5.0 ' # Make sure Balmer lines have adequate S/N
query += 'AND h_beta_flux > h_beta_flux_err*5.0 '
query += 'AND d4000_n_sub > d4000_n_sub_err*5.0 '
query += 'AND h_alpha_flux_err > 0 ' #  Consistency check
query += 'AND h_beta_flux_err > 0 '
query += 'AND h_alpha_flux > 2.7*h_beta_flux ' #  Ensures proper continuum subtraction
query += 'AND lgm_tot_p50 > 0 '
query += 'AND sigma_balmer < 509.55 ' #  Exclude the broad line emitting galaxies
query += 'AND s.class = \'GALAXY\' '  #  Looking for galaxies, not stars
query += 'AND s.zwarning = 0 '
#print (query) # useful for debugging, remove first # to uncomment

# send query to SDSS
gals = SDSS.query_sql(query,data_release=16)

# check data table that we got from the data base
gals

plate,fiberid,mjd,ra,dec,z,zwarning,h_beta_flux,h_alpha_flux,h_beta_flux_err,h_alpha_flux_err,bptclass,lgm_tot_p50,lick_hd_a_sub,d4000_n_sub,lick_hd_a_sub_err,d4000_n_sub_err,petror90_r,u,g
int64,int64,int64,float64,float64,float64,int64,float64,float64,float64,float64,int64,float64,float64,float64,float64,float64,float64,float64,float64
331,98,52368,180.71755,-3.4344455,0.04719325,0,60.89765,322.8908,3.126591,5.579741,1,10.51111,2.955382,1.324214,1.823418,0.04606323,7.548193,19.00633,17.36687
333,58,52313,185.17421,-3.373588,0.06113762,0,78.35414,273.3313,2.27171,3.133114,1,9.952115,4.971385,1.262137,0.9581501,0.02330074,5.6225,18.45313,17.33067
335,133,52000,189.02828,-3.5307497,0.07000943,0,64.61208,270.9374,2.779556,3.905551,1,10.15141,3.738568,1.302683,0.878454,0.01997062,5.764417,18.21997,17.10905
330,224,52370,177.71634,-2.5360534,0.08552437,0,49.77313,172.5872,2.531304,3.046535,1,10.0527,4.028034,1.186684,1.35154,0.02852155,8.153114,18.64158,17.56788
339,351,51692,195.09922,-2.2265093,0.08655296,0,59.38163,272.6566,2.434419,3.459759,1,10.11765,5.875441,1.2468,0.9563492,0.02136064,5.353164,19.12785,17.92694
338,266,51694,194.36721,-2.7991393,0.0812414,0,53.32957,298.4123,2.572135,3.475201,1,10.43292,3.502567,1.444381,1.348861,0.03761733,5.09828,19.44892,18.08693
496,201,51988,201.09803,65.406912,0.09003706,0,166.3421,661.0942,3.523089,6.581434,1,10.3159,3.920924,1.10561,1.05055,0.01907656,8.595096,18.40165,17.44063
445,483,51873,129.15936,50.929651,0.0446375,0,175.1524,768.1199,4.746469,9.618892,1,10.69503,1.48509,1.461214,0.4140134,0.01053269,11.15545,17.42341,15.86291
438,43,51884,121.76274,45.676386,0.05022486,0,92.67487,405.7007,2.740864,4.628186,1,10.36949,4.065806,1.351605,0.7992107,0.02143333,8.054012,17.59638,16.32443


### Dereddening the Spectrum

In order to determine the star formation rate (SFR) in each galaxy, we need to get the H $\alpha$ luminosity L(H$\alpha$) emitted from each galaxy. The SQL query gives us the observed flux F(H$\alpha$), but this doesn’t account for dust absorbing some of this light as it travels from a galaxy to the point of observation. Fortunately, we can correct for this “extinction” by making use of the fact that dust preferentially absorbs shorter wavelength light to “deredden” the spectrum. In ideal conditions without dust, we can assume that
F(H$\alpha$)/F(H$\beta$) = 2.86 in star forming regions. Let’s deredden the H$\alpha$ flux.

In [4]:
# Deredden emission lines according to Calzetii+ 2000
# wavelength supplied in angstroms
# wl : wavelenght
# kwl : wavenumber

def deredden(lum,wl,Hb,Ha):
    if (wl >= 6300 and wl <= 22000):
        wl = wl/1e4 # convert to microns
        kwl = 2.659*(-1.857+1.040/wl)+4.05
    elif (wl >= 1200 and wl < 6300):
        wl = wl/1e4 # convert to microns
        kwl = 2.659*(-2.156+1.509/wl-0.198/wl**2+0.011/wl**3)+4.05
    else:
        print ('Wavelength of range')

    # Using k-values calc as above
    kHa = 3.326
    kHb = 4.598

    ebmv = 2.5/(kHb-kHa)*np.log10(Ha/Hb/2.86) # 2.86 for SF; 3.1 for AGN
    lum_dr = lum*10**(0.4*kwl*ebmv)

    return lum_dr

####

Halpha_flux = gals['h_alpha_flux']
Hbeta_flux = gals['h_beta_flux']

#The actual H alpha flux
Halpha_flux_dr = deredden(Halpha_flux,6563,Hbeta_flux,Halpha_flux)

According to the GalSpecLine table, the line flux is returned as 10$^{-17}$ erg/s/cm$^2$, so we first need to multiply our dereddened F(H) by 10$^{-17}$ to get actual flux value. Luminosity has units of energy / second, and an erg is a unit of energy.



In [5]:
flux = Halpha_flux_dr * 1e-17 #Actual flux

1) Calcualte the distance to the galaxies in the sample in cm. Make sure to clarify which hubble constant you are using.

You can convert the redshift (z) to distance (d) using the following relation:  
$$z=H_0 \frac{d}{c}$$

For the Hubble Constant you can use $H_0$=70 (km /s)/Mpc, and c is the speed of light.

2) Convert the H$\alpha$ flux, $F(H_\alpha)$, into H$\alpha$ luminosity, $L(H_\alpha)$. The Luminosity is given by the following equation: $$L(H_\alpha)= 4 \pi d^2 F(H_\alpha)$$

d is the distance to the galaxy.

We can now use the L(H$\alpha$) to determine the star formation rate (SFR) of the galaxies in our sample. The star formation rate is how many new stars a galaxy forms per year in units of solar mass/year (M$_{\odot}$/yr). To calcualte the SFR we can use the following conversion:

$$SFR = \frac{L(H\alpha)}{1.26 \times 10^{41} ergs \ s^{-1}}$$

3) Calcualte the star formation rate as $log_{10}(SFR)$. Make a histogram of $log_{10}(SFR)$. How is the distribution of the star formation rate?  

### The Star Forming Main Sequence

A common measure of galaxy mass is its stellar mass M$_{\star}$ in units of solar mass M$_{\odot}$, however a galaxy’s mass can be defined in many ways. We can roughly envision that M$_{\star}$ is proxy for galaxy size since larger galaxies typically contain more stars.

4) Make a scatter plot of $log_{10}(SFR)$ vs $log_{10}(M_{\star})$. We have $log_{10}(M_{\star})$ provided in the SDSS data as: gals['lgm_tot_p50']. Plot $log_{10}(M_{\star})$ on the x-axis.

**Based on the plot, how does the SFR correlate with the stallar mass M$_{\star}$?**


The expected correlation is the following:

$$log_{10}(SFR) = log_{10}M_{\star} \times 1.292 - 12.31$$

5) Make the same plot $log_{10}(SFR)$ vs $log_{10}(M_{\star})$ adding the expected correlation. What can you conclude form this plot? Are there many galaxies that are above the correlation?  

6) <b>What kind of galaxies are the ones with high star formation rates?

Are these spirals or ellipticals?

Do these galaxies look blue or red?</b>

To answere this, download a sample of images for 16 galaxies for which $log_{10}(SFR)$ is smaller than the 25th percentile of the data.

You can use the code below to get the image cutouts.

In [6]:
#
# Function to display images
#
def get_images(gal_array):
    # set thumbnail parameters
    width=200           # image width
    height=200          # height
    pixelsize=0.396     # image scale
    plt.figure(figsize=(15, 15))   # display in a 4x4 grid
    subPlotNum = 1

    i = 0
    new_gals = gal_array
    nGalaxies = len(new_gals)
    for index in range(0,nGalaxies):           # iterate through rows in the DataFrame
        i = i + 1
        if i<17:
            print('Getting image of '+str(i)+' of '+str(nGalaxies)+'...')
            if (i == nGalaxies):
                print('Plotting images...')
            scale=2.0 * new_gals['petror90_r'] / pixelsize / width

            pos = coords.SkyCoord(new_gals['ra'][index], new_gals['dec'][index], unit=u.deg, frame='icrs')

            cutoutbaseurl = 'http://skyservice.pha.jhu.edu/DR12/ImgCutout/getjpeg.aspx'
            query_string = urlencode(dict(ra=new_gals['ra'][index], dec=new_gals['dec'][index],width=width, height=height, scale=scale[i]))
            url = cutoutbaseurl + '?' + query_string

            # this downloads the image
            image_name = 'cutout_'+str(index)+'.jpg'
            urlretrieve(url, image_name)

            # Display the image
            img = imread(image_name)

            plt.subplot(4,4,subPlotNum)
            subPlotNum += 1
            plt.imshow(img)                               # show images in grid
            plt.title(index)                              # show the object identifier (objId) above the image.

######


7) <b>What kind of galaxies are the ones with low star formation rates?

Are these spirals or ellipticals?

Do these galaxies look blue or red? </b>

To answere this, download a sample of images for 16 galaxies for which $log_{10}(SFR)$ is larger than the 75th percentile of the data.

8) **Do yuo see a difference between the low and the high star formation rate sample?**

Let's investigate the colours of these galaxies. Make a histogram of the u-g coulour.

**Does the histogram look the same as the u-g colour histogram in the tutorial notebook?**  



9) There are galaxies that form stars in an extreme way. These galaxies are called **starbusrt galaxies**.

**How do the starburst galaxies look like?**

To answere this, download a sample of images for 16 galaxies for which $log_{10}(SFR)$ is larger than the 99th percentile of the data.



### The specific star formation rate

Typically more massive galaxies have more gas and are able to form stars. This means that star formation rate alone is not a good method in evaluating how activly a galaxy is forming stars and growing. A better parameter is, if we normalise the star formation rate with the stellar mass. This is called **specific star formation rate (sSFR)**. It is defined as

$$sSFR = \frac{SFR}{M_{\star}}$$

and represents ratio of current star formation to past star formation, or the growth rate of stellar mass in a galaxy. We can calcualte the sSFR to our dataframe to explore the metric in detail.

9) Calcualte the $log_{10}(sSFR)$ and make a scatter plot of $log_{10}M_{\star}$ on the x-axis and $log_{10}(sSFR)$ on the y-axis. Make sure to exclude outliers from the plot. For this you can use ax.set_ylim().

**Does this plot show the same correlation as the $log_{10}(SFR)$ - $log_{10}M_{\star}$ plot?**

**Are the high mass galaxies or the low mass galaxies more efficient at formaing stars?**

10) **What kind of galaxies have the highest sSFR?**

These galaxies form stars in an extreme way. These galaxies are called **starbusrt galaxies**.

**How do the starburst galaxies look like?**

To answere this, download a sample of images for 16 galaxies for which $log_{10}(sSFR)$ is larger than the 99.9th percentile of the data.